# Transformer Tutorial 3：多模态医学图像分类（超详细新手版）

> 目标：手把手带你完成一个**可运行**的多模态医学图像分类项目。
>
> - **模态 1：医学影像**（示例用 MedMNIST 的胸部 X 光）
> - **模态 2：结构化/文本信息**（示例用“年龄+性别+症状描述”的简化文本）
>
> 你将学会：
> 1. 如何准备数据（自己的数据要怎么放）
> 2. 如何构建多模态数据集
> 3. 如何搭建 Transformer 模型（图像 + 文本）
> 4. 如何训练、评估、保存模型
> 5. 如何进行预训练与微调的思路

**阅读方式建议**：从上到下顺序执行，每一步我都写了“这是在做什么”的解释。


## 0. 目录与文件结构（新手必读）

本教程默认你在项目根目录运行（例如 `Code-for-medAI/`）。建议你把这个 Notebook 放在：

```
Code-for-medAI/
  tutorials/
    transformer_tutorial3_multimodal.ipynb  ← 本文件
```

### 你的**自有数据**应该怎么放？

我们推荐如下结构（把图片和文本/表格信息整理到一起）：

```
my_dataset/
  images/
    train/
      class0/xxx.png
      class1/yyy.png
    val/
      class0/...
      class1/...
  metadata.csv   # 每一行对应一张图（文件名、年龄、性别、症状等）
```

`metadata.csv` 示例（用逗号分隔）：

```
image_id,age,sex,complaint,label
xxx.png,45,M,咳嗽发热,0
yyy.png,70,F,胸闷气短,1
```

> **小白提示**：如果你还没有自己的真实数据，可以先运行本教程的“示例数据”流程。


## 1. 环境准备（安装依赖）

> 如果你是第一次运行，需要安装一些库。
> 下面的命令可以复制到终端里执行。

我们主要用到：
- `torch` / `torchvision`：深度学习框架
- `medmnist`：医学数据集（示例使用）
- `pandas`：处理表格数据

**注意**：在没有网络的环境下，`pip` 可能无法下载。那就先跳过安装。


In [ ]:
# 如果需要安装依赖，取消下面注释并运行
# !pip install torch torchvision medmnist pandas scikit-learn tqdm


## 2. 导入库（只是准备，不会立刻训练）


In [ ]:
import os
from dataclasses import dataclass
from typing import List, Dict

import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import vit_b_16

from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm


## 3. 准备示例数据（MedMNIST）

我们用 **MedMNIST 的 ChestMNIST** 做演示。它是医学影像分类的标准小数据集。

> 这一步会**自动下载数据**（若网络不可用就会失败）。
> 如果你有自己的数据，请跳到第 4 节。


In [ ]:
from medmnist import ChestMNIST

# 这里下载并加载示例数据集
train_dataset_raw = ChestMNIST(split="train", download=True)
val_dataset_raw = ChestMNIST(split="val", download=True)


### 3.1 为示例数据创建“伪文本信息”

ChestMNIST 本身只有图片和标签，没有文本信息。
为了演示“多模态”，我们**人为构造**一些简单文本：
- 随机年龄
- 随机性别
- “症状描述”（用标签映射）

这样你就能看到**图像 + 文本**一起训练。


In [ ]:
import random

symptom_map = {
    0: "无明显异常",
    1: "肺部异常",
}


def build_fake_text(label: int) -> str:
    age = random.randint(18, 90)
    sex = random.choice(["男", "女"])
    symptom = symptom_map.get(label, "未知")
    return f"年龄{age}岁，性别{sex}，症状：{symptom}"


## 4. 文本处理：把中文文本变成模型能理解的数字

对于新手来说，最简单的方法是：
1. 把文本切成“字”或“词”
2. 映射成数字 ID
3. 用 embedding 表示

我们这里用**按字切分**，并用一个最小的“词表”。


In [ ]:
# 1) 先准备一个最简单的字典（包含常见字符）
# 你也可以替换成真正的分词器（如 transformers 的 tokenizer）

SPECIAL_TOKENS = ["<PAD>", "<UNK>"]


def build_vocab(texts: List[str], min_freq: int = 1) -> Dict[str, int]:
    counter = {}
    for t in texts:
        for ch in t:
            counter[ch] = counter.get(ch, 0) + 1
    vocab = {tok: idx for idx, tok in enumerate(SPECIAL_TOKENS)}
    for ch, cnt in counter.items():
        if cnt >= min_freq and ch not in vocab:
            vocab[ch] = len(vocab)
    return vocab


def encode_text(text: str, vocab: Dict[str, int], max_len: int = 50) -> List[int]:
    ids = [vocab.get(ch, vocab["<UNK>"]) for ch in text]
    ids = ids[:max_len]
    if len(ids) < max_len:
        ids += [vocab["<PAD>"]] * (max_len - len(ids))
    return ids


## 5. 构建多模态数据集（图像 + 文本）

我们将“图像”和“文本”合成一个 Dataset：
- `image`：Tensor
- `text_ids`：Tensor
- `label`：Tensor


In [ ]:
@dataclass
class MultiModalConfig:
    image_size: int = 224
    max_text_len: int = 50


class MultiModalDataset(Dataset):
    def __init__(self, raw_dataset, vocab, config: MultiModalConfig, transform=None):
        self.raw_dataset = raw_dataset
        self.vocab = vocab
        self.config = config
        self.transform = transform

    def __len__(self):
        return len(self.raw_dataset)

    def __getitem__(self, idx):
        image, label = self.raw_dataset[idx]
        label = int(label)

        # 生成伪文本
        text = build_fake_text(label)
        text_ids = encode_text(text, self.vocab, self.config.max_text_len)

        # 图像转换
        if self.transform:
            image = self.transform(image)

        return {
            "image": image,
            "text_ids": torch.tensor(text_ids, dtype=torch.long),
            "label": torch.tensor(label, dtype=torch.long),
        }


### 5.1 生成词表和 DataLoader


In [ ]:
config = MultiModalConfig()

# 先构造所有文本，生成词表
all_texts = [build_fake_text(int(train_dataset_raw[i][1])) for i in range(len(train_dataset_raw))]
vocab = build_vocab(all_texts)

image_transform = transforms.Compose([
    transforms.Resize((config.image_size, config.image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
])

train_dataset = MultiModalDataset(train_dataset_raw, vocab, config, transform=image_transform)
val_dataset = MultiModalDataset(val_dataset_raw, vocab, config, transform=image_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print("训练样本数：", len(train_dataset))
print("验证样本数：", len(val_dataset))


## 6. 构建多模态 Transformer 模型

模型结构分成三部分：

1. **图像编码器**（Vision Transformer, ViT）
2. **文本编码器**（简单 Transformer Encoder）
3. **融合 + 分类头**（concat 后 MLP 分类）

> 注意：如果你电脑配置低，可以把 ViT 换成 ResNet，或者把 `weights=None`（不下载预训练权重）。


In [ ]:
class TextEncoder(nn.Module):
    def __init__(self, vocab_size: int, embed_dim: int = 128, num_heads: int = 4, num_layers: int = 2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, batch_first=True)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

    def forward(self, text_ids):
        # text_ids: [B, T]
        x = self.embedding(text_ids)  # [B, T, D]
        x = self.encoder(x)
        # 取平均池化作为文本特征
        return x.mean(dim=1)


class MultiModalTransformer(nn.Module):
    def __init__(self, num_classes: int, vocab_size: int):
        super().__init__()
        # 图像编码器（ViT）
        self.image_encoder = vit_b_16(weights=None)
        image_feature_dim = self.image_encoder.heads.head.in_features
        self.image_encoder.heads = nn.Identity()

        # 文本编码器
        self.text_encoder = TextEncoder(vocab_size=vocab_size)
        text_feature_dim = 128

        # 融合与分类
        self.classifier = nn.Sequential(
            nn.Linear(image_feature_dim + text_feature_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes),
        )

    def forward(self, image, text_ids):
        img_feat = self.image_encoder(image)
        txt_feat = self.text_encoder(text_ids)
        fused = torch.cat([img_feat, txt_feat], dim=1)
        return self.classifier(fused)


## 7. 训练准备

我们准备：
- loss 函数
- 优化器
- 训练循环


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_classes = len(train_dataset_raw.labels[0]) if hasattr(train_dataset_raw, "labels") else 2
model = MultiModalTransformer(num_classes=num_classes, vocab_size=len(vocab)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


## 8. 训练模型（一步一步说明）

我们写一个最清晰的训练函数：
- 逐 batch 前向传播
- 计算 loss
- 反向传播更新


In [ ]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0

    for batch in tqdm(loader, desc="Training"):
        images = batch["image"].to(device)
        text_ids = batch["text_ids"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(images, text_ids)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)


## 9. 验证 / 评估

评估时要：
- 关闭梯度
- 计算准确率和分类报告


In [ ]:
def evaluate(model, loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating"):
            images = batch["image"].to(device)
            text_ids = batch["text_ids"].to(device)
            labels = batch["label"].to(device)

            outputs = model(images, text_ids)
            preds = torch.argmax(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print(classification_report(all_labels, all_preds))
    print("Confusion Matrix:
", confusion_matrix(all_labels, all_preds))


## 10. 开始训练（完整流程）

> 初学者建议先训练 1-2 个 epoch 观察是否跑通。


In [ ]:
EPOCHS = 2
for epoch in range(EPOCHS):
    loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {loss:.4f}")
    evaluate(model, val_loader, device)


## 11. 保存与加载模型

训练完成后保存模型，方便以后继续训练或部署。


In [ ]:
# 保存
torch.save(model.state_dict(), "multimodal_vit.pth")

# 加载（再次使用时）
# model = MultiModalTransformer(num_classes=num_classes, vocab_size=len(vocab))
# model.load_state_dict(torch.load("multimodal_vit.pth", map_location=device))


## 12. 如何进行预训练 & 微调（关键概念）

### 12.1 预训练（Pre-training）
- 意思是：先在**大数据**上训练一个“通用模型”
- 对图像：可以使用 ImageNet 预训练 ViT
- 对文本：可以使用 BERT / RoBERTa 等

### 12.2 微调（Fine-tuning）
- 意思是：把预训练模型用在你的**小数据**上
- 一般流程：
  1. 加载预训练权重
  2. 替换最后一层分类头
  3. 用小学习率训练

### 12.3 在本教程中如何做

你可以这样做：

```python
# 使用预训练 ViT 权重（需要联网下载）
self.image_encoder = vit_b_16(weights="IMAGENET1K_V1")

# 其他步骤不变
```

如果你的文本非常多，可以用 `transformers` 的 BERT：

```python
from transformers import BertModel
self.text_encoder = BertModel.from_pretrained("bert-base-chinese")
```

> 由于教程要“可运行”，我们用最简版本演示。


## 13. 你自己的数据怎么对接？

核心步骤：

1. **整理图像**：放在 `images/train/` 和 `images/val/` 里。
2. **整理文本/表格**：制作 `metadata.csv`（包含 image_id, age, sex, complaint, label）
3. **写一个自定义 Dataset**：
   - 读取图片
   - 读取 CSV 中对应行
   - 生成文本/数值特征

> 你可以参考下面的伪代码：


In [ ]:
class MyCustomDataset(Dataset):
    def __init__(self, csv_path, image_dir, vocab, config, transform=None):
        self.df = pd.read_csv(csv_path)
        self.image_dir = image_dir
        self.vocab = vocab
        self.config = config
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = os.path.join(self.image_dir, row["image_id"])
        image = Image.open(image_path).convert("RGB")

        text = f"年龄{row['age']}岁，性别{row['sex']}，症状：{row['complaint']}"
        text_ids = encode_text(text, self.vocab, self.config.max_text_len)

        if self.transform:
            image = self.transform(image)

        label = int(row["label"])
        return {
            "image": image,
            "text_ids": torch.tensor(text_ids),
            "label": torch.tensor(label),
        }


## 14. 总结

你已经完成了：
✅ 数据准备（示例 + 自己数据结构）
✅ 多模态 Dataset
✅ Transformer 模型（图像 + 文本）
✅ 训练 + 评估
✅ 预训练/微调思路

如果你愿意，我还可以继续帮你扩展：
- 引入真实 BERT
- 引入更复杂的医学文本结构
- 加入 3D 影像（CT/MRI）
- 加入多任务学习（分割 + 分类）
